## pepper data acquisition

### import library

In [2]:
import naoqi as n
import argparse
import sys
import os
import time
import calendar
import cv2
from PIL import Image
import vision_definitions
import numpy as np

nao_ip = '192.168.0.64'
#nao_ip = '192.168.0.25'
port = 9559

motion = n.ALProxy('ALMotion', nao_ip, port)
motion.wakeUp()

True

In [5]:
#motion.rest()

In [3]:
motion.setCollisionProtectionEnabled('LArm', False)
motion.getCollisionProtectionEnabled('LArm')

False

In [4]:
motion.setExternalCollisionProtectionEnabled('LArm',False)
motion.getExternalCollisionProtectionEnabled('LArm')

False

In [7]:
motion.setCollisionProtectionEnabled('Body', False)

RuntimeError: 	ALMotion::setCollisionProtectionEnabled
		ALMotion::setCollisionProtectionEnabled
	The chain name must be "Arms", "LArm" or "RArm".

### initialize module

In [5]:
memory = n.ALProxy('ALMemory', nao_ip, port)
#almodule = n.ALProxy('ALModule', nao_ip, port)

#touch = n.ALModule('ALTouch')
#touch = memory.subscriber('TouchChanged')
touch = n.ALProxy('ALTouch', nao_ip, port)
vision = n.ALProxy('ALVideoDevice', nao_ip, port)
record = n.ALProxy('ALMotionRecorder', nao_ip, port)
video = n.ALProxy("ALVideoDevice", nao_ip, port)
month_dict = dict((month, index) for index, month in enumerate(calendar.month_abbr) if month)
#memory.getDataList()

In [6]:
# MEMORY_VALUE_NAMES is the list of ALMemory values names you want to save.
def recordData(ALMEMORY_KEY_NAMES):
    """ Record the data from ALMemory.
    Returns a matrix of values

    """
    print "Recording data ..."
    data = list()
    start = time.time()
    for range_counter in range(200):
        line = list()
        for key in ALMEMORY_KEY_NAMES:
            value = memory.getData(key)
            line.append(value)
        data.append(line)
        time.sleep(0.01)
    print 'Recording finished! time : ', time.time()-start    
    return data

In [7]:
np.array(data)[1][:5]

NameError: name 'data' is not defined

In [8]:
ALMEMORY_KEY_NAMES = ["Device/SubDeviceList/LShoulderPitch/Position/Actuator/Value",
                      "Device/SubDeviceList/LShoulderRoll/Position/Actuator/Value",
                      "Device/SubDeviceList/LElbowYaw/Position/Actuator/Value",
                      "Device/SubDeviceList/LElbowRoll/Position/Actuator/Value",
                      "Device/SubDeviceList/LWristYaw/Position/Actuator/Value",
                      "Device/SubDeviceList/LShoulderPitch/Position/Sensor/Value",
                      "Device/SubDeviceList/LShoulderRoll/Position/Sensor/Value",
                      "Device/SubDeviceList/LElbowYaw/Position/Sensor/Value",
                      "Device/SubDeviceList/LElbowRoll/Position/Sensor/Value",
                      "Device/SubDeviceList/LWristYaw/Position/Sensor/Value"]

resolution = vision_definitions.kQVGA # (320x240)
colorSpace = 10 #vision_definitions.kYUVColorSpace
fps = 30

videoClient = video.subscribe("pepper_video1", resolution, colorSpace, fps)
prevTime = 0
FPS= 0

output_path = ('/home/beom/beom_ws/pepper_test/pepper_%02d_%02d_%04d.mp4' %(int(month_dict[time.ctime().split()[1]]), int(time.ctime().split()[2]), int(time.ctime().split()[3][:2]+time.ctime().split()[3][3:5])))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), 5, (320,240))

recording_joints = ['LShoulderPitch', 'LShoulderRoll', 'LElbowYaw', 'LElbowRoll', 'LWristYaw']
count = -1
#logging_data = []
joint_data = []
head_data = []
position_data = []

while True:
    if memory.getData("MiddleTactilTouched"):
        motion.setStiffnesses('Head', 0.0)
        print 'Head is touched'
    else : 
        motion.setStiffnesses('Head', 1.0)
       
    if memory.getData('LeftBumperPressed'):
        motion.setStiffnesses('LArm', 0.0)
        print ('stiffness set to %0.2d' %motion.getStiffnesses("LArm")[0])
        #time.sleep(0.5)
        
    if memory.getData('BackBumperPressed'):
        data = recordData(ALMEMORY_KEY_NAMES)
        
    
    if memory.getData('RightBumperPressed'):
        motion.setStiffnesses('LArm', 0.9)
        while True:
            count += 1 
            curTime = time.time()
            naoImage = video.getImageRemote(videoClient)
            #t1 = time.time()

            # Get the image size and pixel array.
            imageWidth = naoImage[0]
            imageHeight = naoImage[1]
            array = naoImage[6]
            image_string = str(bytearray(array))

            im = Image.frombytes("RGB", (imageWidth, imageHeight), image_string)
            #image = numpy.array(im)

            image = cv2.cvtColor(np.array(im), cv2.COLOR_YUV2RGB)#, cv2.COLOR_BGR2RGB)


            #video_fps = vid.get(cv2.CAP_PROP_FPS)
            #out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
            #frame = Image.fromarray(image)
            #image_detect = YOLO().detect_image(frame)
            #result = np.asarray(image_detect)
            out.write(image)
            cv2.putText(image, 'FPS : %0.3f' %FPS, (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            #result = cv2.cvtColor(numpy.array(im), cv2.COLOR_YUV2RGB)
            cv2.imshow("pepper_test",image)
            
            if count < 200:
                #motion.setAngles(recording_joints, np.array(angle_lists).T[count].tolist(), 0.1)
                motion.setAngles(recording_joints, data[count][5:], 0.3)
                joint_data.append(motion.getAngles('LArm', False)[:5])
                head_data.append(motion.getAngles('Head', False))
                position_data.append(motion.getPosition('LWristYaw', 0, False))
                #print np.array(angle_lists).T[count].tolist()
            else:
                pass

            sec = curTime - prevTime
            FPS = 1 / (sec)
            prevTime = curTime


            if cv2.waitKey(1) & 0xFF == ord('q') or count == 300 :
                video.unsubscribe(videoClient)
                out.release()
                cv2.destroyAllWindows()
                count = -1
                break

# Register a Generic Video Module




Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is touched
Head is 

RuntimeError: 	ALMemory::getData
	Session closed

In [15]:
print 'joint : ' , np.array(joint_data).shape
print 'head_data : ' , np.array(head_data).shape
print 'position_data : ', np.array(position_data).shape

joint :  (200, 5)
head_data :  (200, 2)
position_data :  (200, 6)


In [17]:
joint_data = np.array(joint_data)

In [19]:
np.save('/home/beom/beom_ws/pepper_data_acquisition/joint_data', joint_data)
np.save('/home/beom/beom_ws/pepper_data_acquisition/head_data', head_data)
np.save('/home/beom/beom_ws/pepper_data_acquisition/position_data', position_data)


In [ ]:
time.ctime().split()

In [37]:
motion.getStiffnesses('LArm')

[0.8999999761581421,
 0.8999999761581421,
 0.8999999761581421,
 0.8999999761581421,
 0.8999999761581421,
 0.8999999761581421]

In [19]:
motion.setStiffnesses('LArm', 0.3)
for i in range(100):
    motion.setAngles(recording_joints, data[i][5:], 0.1)
    time.sleep(0.5)

--- ---

In [ ]:
#motion.wakeUp()
motion.angleInterpolation(recording_joints, angle_lists, time_lists, isabsolute)

In [22]:
recording_joints
data[90][5:]

[1.3468351364135742,
 0.2837865352630615,
 -1.2593982219696045,
 -0.6994953155517578,
 0.07819199562072754]

In [28]:
motion.setAngles(recording_joints, data[90][:5], 0.1)

In [ ]:
resolution = vision_definitions.kQVGA # (320x240)
colorSpace = 10 #vision_definitions.kYUVColorSpace
fps = 30

videoClient = video.subscribe("pepper_video1", resolution, colorSpace, fps)
prevTime = 0
FPS= 0

output_path = ('/home/mmmil-hw/Ubuntu/pepper_test/pepper_%02d_%02d_%04d.mp4' %(int(month_dict[time.ctime().split()[1]]), int(time.ctime().split()[2]), int(time.ctime().split()[3][:2]+time.ctime().split()[3][3:5])))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), 5, (320,240))

while True:
    count += 1 
    curTime = time.time()
    naoImage = video.getImageRemote(videoClient)
    #t1 = time.time()

    # Get the image size and pixel array.
    imageWidth = naoImage[0]
    imageHeight = naoImage[1]
    array = naoImage[6]
    image_string = str(bytearray(array))

    im = Image.frombytes("RGB", (imageWidth, imageHeight), image_string)
    #image = numpy.array(im)

    image = cv2.cvtColor(np.array(im), cv2.COLOR_YUV2RGB)#, cv2.COLOR_BGR2RGB)


    #video_fps = vid.get(cv2.CAP_PROP_FPS)
    #out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
    #frame = Image.fromarray(image)
    #image_detect = YOLO().detect_image(frame)
    #result = np.asarray(image_detect)
    out.write(image)
    cv2.putText(image, 'FPS : %0.3f' %FPS, (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    #result = cv2.cvtColor(numpy.array(im), cv2.COLOR_YUV2RGB)
    cv2.imshow("pepper_test",image)
    if count < 200:
        motion.setAngles(recording_joints, np.array(angle_lists).T[count].tolist(), 0.1)
        print np.array(angle_lists).T[count].tolist()
    else:
        pass

    sec = curTime - prevTime
    FPS = 1 / (sec)
    prevTime = curTime


    if cv2.waitKey(1) & 0xFF == ord('q'): or 
        video.unsubscribe(videoClient)
        out.release()
        cv2.destroyAllWindows()
        break

In [ ]:
recording_joints = ['LShoulderPitch', 'LShoulderRoll', 'LElbowYaw', 'LElbowRoll', 'LWristYaw']
while True:
    if memory.getData('LeftBumperPressed'):
        motion.setStiffnesses('LArm', 0.3)
        print 'stiffness set to' + m.getstiffnesses("LArm")
        
    if memory.getData('BackBumperPressed'):
        data = recordData(ALMEMORY_KEY_NAMES)
#stiffness off -> record data(expert demonstration) -> replay the motion + video + logging the replay motion 


In [ ]:
motion.setStiffnesses('LArm', 1.0)

recording_joints = ['LShoulderPitch', 'LShoulderRoll', 'LElbowYaw', 'LElbowRoll', 'LWristYaw']
time_list = np.arange(0,30,0.3)[1:].tolist() # float
#time_lists = np.arange(1,100,1)
angle_lists = np.array(data)[:,:5].T.tolist()
isabsolute = True

time_lists = []
for i in range(5):
    time_lists.append(time_list) 
    
#motion.wakeUp()
motion.angleInterpolation(recording_joints, angle_lists, time_lists, isabsolute)